## WhiteBox Error and Sensitivity Analysis on Wine Quality Data

Goals of tutorial include:
* [Importing wine quality dataset](#wine_quality)
* [Handling categorical features inherent within data by using dummy variables and pandas categorical datatype](#categorical)
* [Build model](#model)
* [Deploying WhiteBoxError graphics](#wbox_error)
* [Deploying WhiteBoxSensitivity graphics](#wbox_sensitivity)

In [11]:
# remove when actual package
%load_ext autoreload

%autoreload 2

import os
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

module_path = os.path.abspath('/projects/us_eminence/WhiteBox_Production')
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from whitebox import utils
from whitebox.WhiteBox import WhiteBoxError

### Import wine quality dataset <a id=wine_quality><a>
Perform basic exploratory data analysis to better understand what types of columns are available

In [13]:
df = pd.read_csv('./datasets/winequality.csv')

In [14]:
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,quality,Type,AlcoholContent
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,5,Red,Low
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,5,Red,Low
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,5,Red,Low
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,6,Red,Low
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,5,Red,Low


In [17]:
df.dtypes # it looks like most of our columns are numeric, with the exception of Type and AlcoholContent

fixed.acidity           float64
volatile.acidity        float64
citric.acid             float64
residual.sugar          float64
chlorides               float64
free.sulfur.dioxide     float64
total.sulfur.dioxide    float64
density                 float64
pH                      float64
sulphates               float64
quality                   int64
Type                     object
AlcoholContent           object
dtype: object

In [20]:
df.groupby('AlcoholContent')['fixed.acidity'].count() # most of our data resides in low/medium alcohol content

AlcoholContent
High       852
Low       2832
Medium    2813
Name: fixed.acidity, dtype: int64

In [22]:
df.groupby('Type')['fixed.acidity'].count() # and most of our data is white wine

Type
Red      1599
White    4898
Name: fixed.acidity, dtype: int64

### Handling categorical data <a id=categorical><a>

In [27]:
# we need to convert the categorical columns to the special pandas categorical datatype

# first need to identify these string columns from before
string_cols = df.select_dtypes(include = ['O'])

# convert these into categorical datatype
for cat in string_cols:
    df[cat] = pd.Categorical(df[cat])

In [29]:
df.dtypes # great - now they are categorical datatypes

fixed.acidity            float64
volatile.acidity         float64
citric.acid              float64
residual.sugar           float64
chlorides                float64
free.sulfur.dioxide      float64
total.sulfur.dioxide     float64
density                  float64
pH                       float64
sulphates                float64
quality                    int64
Type                    category
AlcoholContent          category
dtype: object

### Build model <a id=model><a>

In [30]:
# we need to define what our dependent variable is
ydepend = 'quality' # we are going to treat quality as a continuous variable for the purposes of the tutorial

# subset the training data
x_train = df.loc[:, df.columns != ydepend].copy(deep = True)

# convert these into their categorical codes using utility function convert_categorical_indpendent
x_train = utils.convert_categorical_independent(x_train)

# pull out the dependent variable column
y_train = df.loc[:, ydepend]

In [31]:
# build the model
modelobj = RandomForestRegressor()

modelobj.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [32]:
df.head(1)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,quality,Type,AlcoholContent
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,5,Red,Low


### Create WhiteBoxError <a id=wbox_error><a>

In [33]:
# specify featuredict as a subset of columns we want to focus on
featuredict = {'fixed.acidity': 'FIXED ACIDITY',
               'Type': 'TYPE',
               'quality': 'SUPERQUALITY',
               'AlcoholContent': 'AC',
               'sulphates': 'SULPHATES',
              'volatile.acidity': 'VOLATILE ACIDITY',
              'residual.sugar': 'RESIDUAL SUGAR',
              'free.sulfur.dioxide': 'FREE SULFUR DIOXIDE'}

# specify the groupby variables
groupbyvars = ['Type']

# instantiate wbox error
WB = WhiteBoxError(modelobj = modelobj,
                   model_df = x_train,
                   ydepend= ydepend,
                   cat_df = df,
                   groupbyvars = groupbyvars,
                   featuredict = featuredict,
                   verbose=2)

2018-02-01 15:29:20,557:[WhiteBox.py:111 -             __init__()]
                                          INFO:
Logger started....


In [34]:
# run wbox error
WB.run()

2018-02-01 15:29:33,283:[WhiteBox.py:142 -              predict()]
                                          INFO:
Creating predictions using modelobj.
                        
Modelobj class name: RandomForestRegressor
2018-02-01 15:29:33,315:[WhiteBox.py:150 -              predict()]
                                          INFO:
Assigning predictions to instance dataframe
2018-02-01 15:29:33,319:[WhiteBox.py:180 -                  run()]
                                          INFO:
Running main program. Iterating over columns and applying functions depednent on datatype
2018-02-01 15:29:33,321:[WhiteBox.py:188 -                  run()]
                                          INFO:
Currently working on column: fixed.acidity
                                
Groupby: Type

2018-02-01 15:29:33,323:[WhiteBox.py:479 -            var_check()]
                                          INFO:
Column determined as continuous datatype, transforming data for continuous column
             

2018-02-01 15:29:33,782:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:33,798:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:33,815:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_f

2018-02-01 15:29:34,113:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (22, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:34,129:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (17, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:34,146:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_f

2018-02-01 15:29:34,444:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (16, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:34,460:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (14, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:34,649:[WhiteBox.py:255 -     continuous_slice()]
                                          INFO:
Creating percentile bins for current continuous group

vartype: Continuous
2018-02-01 15:29:34,946:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:34,957:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (44, 5)
                                         
Col: fixed.acidity
                                         
vartype: Continuous
2018-02-01 15:29:34,968:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate v

...

2018-02-01 15:29:45,751:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (46, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:45,761:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (39, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:45,771:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in

2018-02-01 15:29:45,966:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (17, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:45,978:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (14, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:45,989:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in

2018-02-01 15:29:46,357:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (102, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:46,370:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (110, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:46,384:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values 

2018-02-01 15:29:46,642:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (105, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:46,657:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (88, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:46,672:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values i

2018-02-01 15:29:46,938:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (89, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:46,953:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in transform_function
                                         
Group: Type
                                         
Group Shape: (30, 5)
                                         
Col: free.sulfur.dioxide
                                         
vartype: Continuous
2018-02-01 15:29:46,968:[WhiteBox.py:433 -   transform_function()]
                                          INFO:
"Returning continuous aggregate values in

In [38]:
WB.outputs[0]

{'Data': [{'MSE': 0.06291432145090758,
   'Total': 1599,
   'groupByValue': 'Red',
   'groupByVarName': 'Type'},
  {'MSE': 0.06935892200898233,
   'Total': 4898,
   'groupByValue': 'White',
   'groupByVarName': 'Type'}],
 'Type': 'Accuracy'}

In [36]:
WB.save('../output/example_winequality_error.html')

2018-02-01 15:30:50,278:[WhiteBox.py:295 -                 save()]
                                          INFO:
creating html output for type: html_error
2018-02-01 15:30:50,283:[WhiteBox.py:301 -                 save()]
                                          INFO:
Writing html file out to disk
